## Oppsett

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# sns.set(color_codes=True)
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['font.size'] = 16.0

np.random.seed(1337)

## Importer data

Datasettet har 28480 rader og 30 kolonner (hvorav én er label).

Det første vi gjør er å importere datasettet og splitte attributtene fra klassen til en **X**-matrise og **y**-vektor.

**X** er allerede skalert med `sklearn.metrics.scale`.

In [ ]:
def read_dataset(filename):
    cc = pd.read_csv("data/" + filename)
    X = cc.drop("Class", axis=1)
    y = cc["Class"]
    
    return X, y

X, y = read_dataset("cc_scaled_small.csv")
X.head()

Vi vet ikke hva attributtene betyr, så vi får bare håpe på det beste!

Splitt opp datasettet i treningsdata og testdata og tren en modell. Vi bruker her en `Random Forest`, som bruker en samling tilfeldige Decision Trees for å ta avgjørelser.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1337, stratify=y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

## Oppgave 1: Accuracy

Vi har nå modellens antakelse i `y_pred` og de faktiske klassene i `y_test`.

Finn modellens nøyaktighet (accuracy).

PS: Dette er veldig overkommelig å gjøre for hånd, men du kan også bruke `sklearn.metrics.accuracy_score`.

Ble resultatet som forventet?

## Oppgave 2: Null Accuracy

Finn frekvensen til den vanligste klassen ("Ikke svindel" = 0).

PS: `sklearn.dummy.DummyClassifier`

Hva sier dette om resultatet i Oppgave 1?

## Oppgave 3: Confusion Matrix

Få et bedre overblikk over hvordan modellen gjorde det ved å plotte en confusion matrix for evalueringen.

Funksjonen under lar deg plotte en confusion matrix på en visuelt forståelig måte. Sleng med `normalize=True` for å få relative tall (per rad == klasse).

Det viktige her er å få en confusion matrix du forstår, så dersom funksjonen under ikke fungerer for deg er det bare å finne en annen måte å gjøre det på!

PS: `sklearn.metrics.confusion_matrix`

In [ ]:
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

import itertools

sns.set_style("white")

def plot_confusion_matrix(cm,
                          normalize=False,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    classes = ["Ikke svindel", "Svindel"]
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes, rotation=45)

    fmt = '.4f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Faktisk klasse')
    plt.xlabel('Antatt klasse')

Hvilke tall er de viktigste her?

Hvorfor?

## Oppgave 4: F-measure

Finn modellens Precision, Recall og F1-score.

PS: Fortsatt `sklearn.metrics`..

Hvilket av disse målene er nyttigst?

## Oppgave 5: AUROC

Regn ut Area Under ROC

PS: `sklearn.metrics.roc_auc_score`

Hva sier dette om `recall` vi fant tidligere? Kan den forbedres uten videre?

## Oppgave 6: Cross-validation

Evaluer modellen med cross-validated recall og undersøk hvor bra modellen gjorde det i hver iterasjon.

PS: `sklearn.model_selection.cross_val_score`

In [ ]:
for i, score in enumerate(scores):
    print "Iterasjon {}: {:.4f}".format(i, scores[i])


print "\nSnitt: {:.4f}".format(scores.mean())

Merk hvor varierende modellen gjør det; Hvordan vi splitter kan altså ha mye å si for resultatet!

## Oppgave 7: Hyperparameter Optimization

Finn en bedre modell ved å initialisere modellen med forskjellige verdier før trening.

In [ ]:
params = { 'n_estimators': range(5, 100)
         , 'criterion': ["gini", "entropy"]
         , 'min_samples_split': range(2, 11)
         , 'min_samples_leaf': range(1, 11)
         , 'max_features': ['sqrt', 'log2'] + range(1, 29)
         , 'max_depth': range(1, 100)
         }

Husk at hele treningsløpet må skje fra bunn hver gang du evaluerer for å forsøke å unngå bias. Du klarer ca. 20 skoger per minutt (hvis du benytter alle kjernene du har), så ikke vær for ambisiøs!

PS: Se på `sklearn.model_selection.GridSearchCV` og `sklearn.model_selection.RandomizedSearchCV`.

## Oppgave 8: Valider på hold-out

Sjekk hvordan den beste modellen du fant i oppgave 7 gjør det på et annet datasett.

`cc_scaled_holdout.csv` er 9 ganger større, så i en realistisk situasjon ville vi trent på litt mer og brukt den ferdigtrente modellen på holdout til slutt. Her kan du sjekke hva du får ved cross-validation av modellen din på hele holdout-settet.

In [ ]:
XX, yy = read_dataset("cc_scaled_holdout.csv")

Er dette noe bedre enn en "dum" (uoptimert) Random Forest?